In [2]:
import requests
import os 
import json
import numpy as np 
import pandas as pd
from datetime import datetime, timedelta
from pprint import pprint 
import pytz
import seaborn as sb
import schedule

from datetime import datetime
from datetime import timedelta
from datetime import date
from sklearn.preprocessing import LabelEncoder
import time
from PIL import Image
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor, as_completed

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
pd.options.mode.chained_assignment = None 
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, RFE, mutual_info_classif
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import asyncio
import aiohttp
import nest_asyncio

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:
#Standings on December 24th or equivalent 

# Define the range of years
start_year = 2000
end_year = 2023
base_url = 'https://api-web.nhle.com/v1/standings/'

# Initialize December 23rd dates
december_23_dates = [
    f"{year}-12-24"
    for year in range(start_year, end_year + 1)
    if year not in [2012, 2020]
]

# Initialize DataFrame to store standings data
december_standings = pd.DataFrame()

# Loop through each December 23rd date
for dec_date in december_23_dates:
    api_url = f"{base_url}{dec_date}"
    response = requests.get(api_url)
    if response.status_code == 200:
        response_json = response.json()

        # Normalize standings data (adjust this based on the API response structure)
        if 'standings' in response_json:
            standings = response_json['standings']
            standings_df = pd.json_normalize(standings)

            # Add the December 23rd date to the data
            standings_df['december_date'] = dec_date

            # Concatenate with the main DataFrame
            december_standings = pd.concat([december_standings, standings_df], ignore_index=True)
        else:
            print(f"No standings data available for {dec_date}")
    else:
        print(f"Failed to retrieve data for {dec_date}, status code: {response.status_code}")


december_standings.head()

,conferenceAbbrev,conferenceHomeSequence,conferenceL10Sequence,conferenceName,conferenceRoadSequence,conferenceSequence,date,divisionAbbrev,divisionHomeSequence,divisionL10Sequence,divisionName,divisionRoadSequence,divisionSequence,gameTypeId,gamesPlayed,goalDifferential,goalDifferentialPctg,goalAgainst,goalFor,goalsForPctg,homeGamesPlayed,homeGoalDifferential,homeGoalsAgainst,homeGoalsFor,homeLosses,homeOtLosses,homePoints,homeRegulationPlusOtWins,homeRegulationWins,homeTies,homeWins,l10GamesPlayed,l10GoalDifferential,l10GoalsAgainst,l10GoalsFor,l10Losses,l10OtLosses,l10Points,l10RegulationPlusOtWins,l10RegulationWins,l10Ties,l10Wins,leagueHomeSequence,leagueL10Sequence,leagueRoadSequence,leagueSequence,losses,otLosses,pointPctg,points,regulationPlusOtWinPctg,regulationPlusOtWins,regulationWinPctg,regulationWins,roadGamesPlayed,roadGoalDifferential,roadGoalsAgainst,roadGoalsFor,roadLosses,roadOtLosses,roadPoints,roadRegulationPlusOtWins,roadRegulationWins,roadTies,roadWins,seasonId,shootoutLosses,shootoutWins,streakCode,streakCount,teamLogo,ties,waiversSequence,wildcardSequence,winPctg,wins,placeName.default,teamName.default,teamName.fr,teamCommonName.default,teamAbbrev.default,teamCommonName.fr,placeName.fr,december_date
0,XVW,1.0,7.0,Western,3.0,1.0,2000-12-24,NW,1.0,2.0,Northwest,1.0,1.0,2.0,35.0,29.0,0.828571,79.0,108.0,3.085714,20.0,18.0,42.0,60.0,3.0,0.0,31.0,14.0,10.0,3.0,14.0,10.0,1.0,28.0,29.0,3.0,0.0,12.0,5.0,4.0,2.0,5.0,1.0,14.0,7.0,1.0,7.0,0.0,0.728571,51.0,0.657143,23.0,0.514286,18.0,15.0,11.0,37.0,48.0,4.0,0.0,20.0,9.0,8.0,2.0,9.0,20002001.0,0.0,0.0,W,2.0,https://assets.nhle.com/logos/nhl/svg/COL_ligh...,5.0,29.0,0.0,0.657143,23.0,Colorado,Colorado Avalanche,Avalanche du Colorado,Avalanche,COL,NaN,NaN,2000-12-24
1,XVW,2.0,1.0,Western,4.0,2.0,2000-12-24,CEN,1.0,1.0,Central,2.0,1.0,2.0,31.0,53.0,1.709677,61.0,114.0,3.677419,16.0,29.0,26.0,55.0,1.0,0.0,28.0,13.0,12.0,2.0,13.0,10.0,25.0,23.0,48.0,0.0,0.0,19.0,9.0,8.0,1.0,9.0,2.0,1.0,3.0,2.0,4.0,0.0,0.806452,50.0,0.741935,23.0,0.677419,21.0,15.0,24.0,35.0,59.0,3.0,0.0,22.0,10.0,9.0,2.0,10.0,20002001.0,0.0,0.0,W,2.0,https://assets.nhle.com/logos/nhl/svg/STL_2000...,4.0,30.0,0.0,0.741935,23.0,St. Louis,St. Louis Blues,Blues de St. Louis,Blues,STL,NaN,NaN,2000-12-24
2,XVW,6.0,6.0,Western,1.0,4.0,2000-12-24,CEN,2.0,2.0,Central,1.0,2.0,2.0,38.0,14.0,0.368421,98.0,112.0,2.947368,21.0,4.0,58.0,62.0,8.0,2.0,23.0,10.0,7.0,1.0,10.0,10.0,3.0,23.0,26.0,3.0,0.0,12.0,5.0,2.0,2.0,5.0,12.0,12.0,1.0,3.0,12.0,2.0,0.618421,47.0,0.552632,21.0,0.421053,16.0,17.0,10.0,40.0,50.0,4.0,0.0,24.0,11.0,9.0,2.0,11.0,20002001.0,0.0,0.0,W,2.0,https://assets.nhle.com/logos/nhl/svg/DET_ligh...,3.0,24.0,0.0,0.552632,21.0,Detroit,Detroit Red Wings,Red Wings de Detroit,Red Wings,DET,NaN,NaN,2000-12-24
3,XVW,7.0,4.0,Western,2.0,3.0,2000-12-24,PAC,2.0,2.0,Pacific,1.0,1.0,2.0,33.0,25.0,0.757576,73.0,98.0,2.969697,15.0,10.0,33.0,43.0,3.0,0.0,22.0,10.0,9.0,2.0,10.0,10.0,6.0,24.0,30.0,3.0,0.0,13.0,6.0,5.0,1.0,6.0,13.0,6.0,2.0,4.0,8.0,0.0,0.696970,46.0,0.636364,21.0,0.545455,18.0,18.0,15.0,40.0,55.0,5.0,0.0,24.0,11.0,9.0,2.0,11.0,20002001.0,0.0,0.0,L,2.0,https://assets.nhle.com/logos/nhl/svg/SJS_1998...,4.0,28.0,0.0,0.636364,21.0,San Jose,San Jose Sharks,Sharks de San Jose,Sharks,SJS,NaN,NaN,2000-12-24
4,XVE,4.0,2.0,Eastern,4.0,1.0,2000-12-24,NE,2.0,1.0,Northeast,2.0,1.0,2.0,34.0,31.0,0.911765,77.0,108.0,3.176471,18.0,17.0,43.0,60.0,4.0,0.0,25.0,11.0,11.0,3.0,11.0,10.0,12.0,19.0,31.0,2.0,0.0,14.0,6.0,6.0,2.0,6.0,5.0,3.0,11.0,5.0,9.0,0.0,0.647059,44.0,0.558824,19.0,0.558824,19.0,16.0,14.0,34.0,48.0,5.0,0.0,19.0,8.0,8.0,3.0,8.0,20002001.0,0.0,0.0,L,1.0,https://assets.nhle.com/logos/nhl/svg/OTT_1997...,6.0,27.0,0.0,0.558824,19.0,Ottawa,Ottawa Senators,Sénateurs d'Ottawa,Senators,OTT,Sénateurs,NaN,2000-12-24


In [4]:
december_standings['gamesPlayed'].mean()

34.838810641627546

In [5]:
#35% of the season in
def lockout_season():
    try: 
        api_url = "https://api-web.nhle.com/v1/standings/2013-02-24"
        response = requests.get(api_url )
        content = json.loads(response.content)


        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # The response content can be accessed using response.text
            response_text = response.text
            #pprint(response_text)
        else:
            print(f"Request failed with status code {response.status_code}")

        json_data = json.loads(response_text)
        standings = json_data['standings']
        lockout_standings =pd.DataFrame(standings)

        # Extract team names
        lockout_standings['team'] = lockout_standings['teamCommonName'].apply(lambda x: x['default'])
        lockout_standings = lockout_standings[['conferenceName', 'conferenceSequence', 'divisionName', 'divisionSequence', 'gamesPlayed', 'goalDifferential', 
                                               'goalDifferentialPctg', 'goalAgainst', 'goalFor', 'goalsForPctg', 'pointPctg', 'points',	'regulationPlusOtWinPctg', 
                                               'regulationPlusOtWins', 'regulationWinPctg', 'regulationWins', 'streakCode', 'streakCount', 'winPctg', 'team']]
        
        # points_per_game = lockout_standings['points'] / lockout_standings['gamesPlayed']
        # games_remaining = 82 - lockout_standings['gamesPlayed']
        # projected_final_points = lockout_standings['points']  + (games_remaining * points_per_game)
        # lockout_standings['projected_points'] = round(projected_final_points)
                # Post-process the standings DataFrame
        if not lockout_standings.empty:
            lockout_standings = lockout_standings.convert_dtypes()
            output_path = r'C:/Users/kjcs2/GitHub/hockey/lockout_christmas.csv'
            lockout_standings.to_csv(output_path, index=False)
            print(f"Thanksgiving standings data saved to {output_path}")
    
    

        return lockout_standings 
    except Exception as e:
        print(f"Error loading standings data: {e}")
        return None
    
    
lockout = lockout_season()

if lockout is not None:
    print(lockout)


else:

    print("No data returned.")

Thanksgiving standings data saved to C:/Users/kjcs2/GitHub/hockey/lockout_christmas.csv
   conferenceName  conferenceSequence divisionName  divisionSequence  \
0         Western                   1      Central                 1   
1         Western                   2      Pacific                 1   
2         Eastern                   1    Northeast                 1   
3         Eastern                   2     Atlantic                 1   
4         Eastern                   4    Northeast                 2   
5         Western                   3    Northwest                 1   
6         Eastern                   5     Atlantic                 2   
7         Eastern                   6    Northeast                 3   
8         Western                   4      Central                 2   
9         Eastern                   7    Northeast                 4   
10        Western                   5      Central                 3   
11        Western                   6      Centr

In [6]:
def covid_season():
    try: 
        api_url = "https://api-web.nhle.com/v1/standings/2021-03-04"
        response = requests.get(api_url )
        content = json.loads(response.content)


        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # The response content can be accessed using response.text
            response_text = response.text
            #pprint(response_text)
        else:
            print(f"Request failed with status code {response.status_code}")

        json_data = json.loads(response_text)
        standings = json_data['standings']
        covid_standings =pd.DataFrame(standings)

        # Extract team names
        covid_standings['team'] = covid_standings['teamCommonName'].apply(lambda x: x['default'])
        # covid_standings = covid_standings[['conferenceName', 'conferenceSequence', 'divisionName', 'divisionSequence', 'gamesPlayed', 'goalDifferential', 
        #                                        'goalDifferentialPctg', 'goalAgainst', 'goalFor', 'goalsForPctg', 'pointPctg', 'points',	'regulationPlusOtWinPctg', 
        #                                        'regulationPlusOtWins', 'regulationWinPctg', 'regulationWins', 'streakCode', 'streakCount', 'winPctg', 'team']]
        
        # points_per_game = lockout_standings['points'] / lockout_standings['gamesPlayed']
        # games_remaining = 82 - lockout_standings['gamesPlayed']
        # projected_final_points = lockout_standings['points']  + (games_remaining * points_per_game)
        # lockout_standings['projected_points'] = round(projected_final_points)
                # Post-process the standings DataFrame
        if not covid_standings.empty:
            covid_standings = covid_standings.convert_dtypes()
            output_path = r'C:/Users/kjcs2/GitHub/hockey/covid_christmas.csv'
            covid_standings.to_csv(output_path, index=False)
            print(f"Thanksgiving standings data saved to {output_path}")
    
    

        return covid_standings 
    except Exception as e:
        print(f"Error loading standings data: {e}")
        return None
    
    
covid = covid_season()

if covid is not None:
    print(covid)


else:

    print("No data returned.")

Thanksgiving standings data saved to C:/Users/kjcs2/GitHub/hockey/covid_christmas.csv
    conferenceHomeSequence  conferenceL10Sequence  conferenceRoadSequence  \
0                        0                      0                       0   
1                        0                      0                       0   
2                        0                      0                       0   
3                        0                      0                       0   
4                        0                      0                       0   
5                        0                      0                       0   
6                        0                      0                       0   
7                        0                      0                       0   
8                        0                      0                       0   
9                        0                      0                       0   
10                       0                      0                  

In [24]:
#today's standings

api_url = "https://api-web.nhle.com/v1/standings/2024-12-24"
response = requests.get(api_url )
content = json.loads(response.content)


# Check if the request was successful (status code 200)
if response.status_code == 200:
    # The response content can be accessed using response.text
    response_text = response.text
    #pprint(response_text)
else:
    print(f"Request failed with status code {response.status_code}")

json_data = json.loads(response_text)
standings = json_data['standings']
standings_df =pd.DataFrame(standings)

# Extract team names
standings_df['team'] = standings_df['teamCommonName'].apply(lambda x: x['default'])
standings_df = standings_df[["conferenceName", "leagueSequence", "divisionName", "gamesPlayed", "teamLogo", "team", 
                            "winPctg", "conferenceSequence", "date", "divisionSequence", "points", "regulationWins", 
                            "regulationPlusOtWins", "pointPctg", "goalDifferential", "goalFor"]]

points_per_game = standings_df['points'] / standings_df['gamesPlayed']
games_remaining = 82 - standings_df['gamesPlayed']
projected_final_points = standings_df['points']  + (games_remaining * points_per_game)
standings_df['projected_points'] = round(projected_final_points)
current_standings = standings_df
current_standings.head()



,conferenceName,leagueSequence,divisionName,gamesPlayed,teamLogo,team,winPctg,conferenceSequence,date,divisionSequence,points,regulationWins,regulationPlusOtWins,pointPctg,goalDifferential,goalFor,projected_points
0,Western,1,Central,36,https://assets.nhle.com/logos/nhl/svg/WPG_ligh...,Jets,0.694444,1,2024-12-24,1,51,21,25,0.708333,44,133,116.0
1,Western,2,Pacific,34,https://assets.nhle.com/logos/nhl/svg/VGK_ligh...,Golden Knights,0.676471,2,2024-12-24,1,49,20,22,0.720588,24,120,118.0
2,Eastern,3,Metropolitan,37,https://assets.nhle.com/logos/nhl/svg/NJD_ligh...,Devils,0.621622,1,2024-12-24,1,49,22,23,0.662162,33,124,109.0
3,Eastern,4,Metropolitan,34,https://assets.nhle.com/logos/nhl/svg/WSH_seco...,Capitals,0.676471,2,2024-12-24,2,48,20,23,0.705882,36,125,116.0
4,Western,5,Central,35,https://assets.nhle.com/logos/nhl/svg/MIN_ligh...,Wild,0.600000,3,2024-12-24,2,46,17,20,0.657143,8,104,108.0


In [1]:
december_standings['gamesPlayed'].mean()

NameError: name 'december_standings' is not defined

In [20]:

#Season End Dates
# Dictionary of seasons and corresponding end dates
nhl_season_dates = {
    "20002001": "2001-04-08",
    "20012002": "2002-04-14",
    "20022003": "2003-04-06",
    "20032004": "2004-04-04",
    "20052006": "2006-04-18",
    "20062007": "2007-04-08",
    "20072008": "2008-04-06",
    "20082009": "2009-04-12",
    "20092010": "2010-04-11",
    "20102011": "2011-04-10",
    "20112012": "2012-04-07",
    "20122013": "2013-04-27",
    "20132014": "2014-04-13",
    "20142015": "2015-04-11",
    "20152016": "2016-04-10",
    "20162017": "2017-04-09",
    "20172018": "2018-04-08",
    "20182019": "2019-04-06",
    "20192020": "2020-03-12",
    "20202021": "2021-05-19",
    "20212022": "2022-04-29",
    "20222023": "2023-04-13",
    "20232024": "2024-04-18"
}

def final_standings(season_dates):
    try:
        base_url = 'https://api-web.nhle.com/v1/standings/'

        # Initialize DataFrame to store all standings data
        final_standings_all = pd.DataFrame()

        # Loop through each season and its end date
        for season_id, end_date in season_dates.items():
            api_url = f"{base_url}{end_date}"
            response = requests.get(api_url)

            if response.status_code == 200:
                response_json = response.json()

                # Check if standings data exists
                if 'standings' in response_json:
                    final = response_json['standings']
                    final_df = pd.json_normalize(final)

                    # Add season ID and end date to the data
                    final_df['season_id'] = season_id
                    final_df['end_date'] = end_date

                    # Concatenate with the main DataFrame
                    final_standings_all = pd.concat([final_standings_all, final_df], ignore_index=True)
                else:
                    print(f"No standings data available for {end_date}")
            else:
                print(f"Failed to retrieve data for {end_date}, status code: {response.status_code}")

        # Save the DataFrame to a CSV file
        final_standings_all.to_csv("final_standings_all.csv", index=False)
        print("Data saved to 'final_standings_all.csv'.")

        return final_standings_all

    except Exception as e:
        print(f"Error loading final standings: {e}")
        return pd.DataFrame()

# Run the function and store the result
finals = final_standings(nhl_season_dates)

# Load the DataFrame later
final_standings_all_loaded = pd.read_csv("final_standings_all.csv")

# Display the loaded DataFrame
if not final_standings_all_loaded.empty:
    print(final_standings_all_loaded.head())
else:
    print("No data found in the saved file.")


Data saved to 'final_standings_all.csv'.
  clinchIndicator conferenceAbbrev  conferenceHomeSequence  \
0               p              XVW                     2.0   
1               y              XVW                     4.0   
2               z              XVE                     5.0   
3               y              XVE                     1.0   
4               y              XVW                     3.0   

   conferenceL10Sequence conferenceName  conferenceRoadSequence  \
0                    3.0        Western                     1.0   
1                    2.0        Western                     2.0   
2                    1.0        Eastern                     1.0   
3                    2.0        Eastern                     2.0   
4                    1.0        Western                     3.0   

   conferenceSequence        date divisionAbbrev  divisionHomeSequence  \
0                 1.0  2001-04-08             NW                   1.0   
1                 2.0  2001-04-08  

In [21]:
final_standings_all_loaded = final_standings_all_loaded[['conferenceName', 'conferenceSequence', 'divisionName', 'divisionSequence', 'goalDifferential', 'goalDifferentialPctg', 'goalAgainst', 'goalFor',
                                                         'goalsForPctg', 'wins', 'losses','pointPctg', 'points', 'winPctg', 'teamName.default', 'streakCode', 'streakCount', 'season_id']]
final_standings_all_loaded.head()

,conferenceName,conferenceSequence,divisionName,divisionSequence,goalDifferential,goalDifferentialPctg,goalAgainst,goalFor,goalsForPctg,wins,losses,pointPctg,points,winPctg,teamName.default,streakCode,streakCount,season_id
0,Western,1.0,Northwest,1.0,78.0,0.951220,192.0,270.0,3.292683,52.0,16.0,0.719512,118.0,0.634146,Colorado Avalanche,W,1.0,20002001
1,Western,2.0,Central,1.0,51.0,0.621951,202.0,253.0,3.085366,49.0,20.0,0.676829,111.0,0.597561,Detroit Red Wings,W,2.0,20002001
2,Eastern,1.0,Atlantic,1.0,100.0,1.219512,195.0,295.0,3.597561,48.0,19.0,0.676829,111.0,0.585366,New Jersey Devils,W,4.0,20002001
3,Eastern,2.0,Northeast,1.0,69.0,0.841463,205.0,274.0,3.341463,48.0,21.0,0.664634,109.0,0.585366,Ottawa Senators,W,2.0,20002001
4,Western,3.0,Pacific,1.0,54.0,0.658537,187.0,241.0,2.939024,48.0,24.0,0.646341,106.0,0.585366,Dallas Stars,W,2.0,20002001


In [32]:
december_standings = december_standings.rename(columns = {'seasondId':'season_id'})
december_standings.columns

AttributeError: 'NoneType' object has no attribute 'rename'

In [26]:
# Ensure both DataFrames have the necessary columns
if 'teamName.default' in december_standings.columns and 'teamName.default' in final_standings_all_loaded.columns:
    # Perform the merge on teamName.default and season_id
    christmas_predictions = december_standings.merge(
        final_standings_all_loaded,
        on=['teamName.default', 'season_id'],  # Specify the columns to merge on
        how='inner'  # Choose the merge type (inner, left, right, outer)
    )

    # Display the merged DataFrame
    christmas_predictions.head()
else:
    print("The required columns for merging are missing in one or both DataFrames.")


KeyError: 'season_id'

In [11]:
christmas_predictions.head()

NameError: name 'christmas_predictions' is not defined

In [ ]:
christmas_clean = christmas_predictions[[ 'season_id', 'conferenceName_x', 'conferenceSequence_x',  'gamesPlayed', 'goalDifferential_x',
       'goalDifferentialPctg_x', 'goalAgainst_x', 'goalFor_x',
       'goalsForPctg_x', 'pointPctg_x', 'points_x', 'streakCode_x',
       'streakCount_x', 'winPctg_x', 'teamName.default', 'teamAbbrev.default',
       'conferenceSequence_y', 
       'goalDifferential_y', 'goalDifferentialPctg_y', 'goalAgainst_y',
       'goalFor_y', 'goalsForPctg_y', 'pointPctg_y',
       'points_y', 'winPctg_y','streakCode_y', 'streakCount_y']]
christmas_clean.head()

In [ ]:
christmas_clean=christmas_clean.rename(columns = {'conferenceName_x': 'conference',
        'conferenceSequence_x': 'thanksgiving',
        'conferenceSequence_y': 'season_end',
        'goalDifferential_x':'tx_goal_diff',
        'goalDifferential_y': 'end_goal_diff',
        'goalDifferentialPctg_x': 'tx_goal_diff_pct',
        'goalDifferentialPctg_y': 'end_goal_diff_pct',
        'goalAgainst_x':'tx_GA',
        'goalAgainst_y': 'end_GA',
        'goalFor_x': 'tx_GF',
        'goalFor_y':'end_GF',
        'goalsForPctg_x':'tx_gf_pctg',
        'goalsForPctg_y': 'end_gf_pctg',
        'points_x':'tx_points',
        'points_y':'end_points',
        'pointPctg_x': 'tx_points_pct',
        'pointPctg_y': 'end_points_pct',
        'streakCode_x': 'tx_streak_code',
        'streakCount_x': 'tx_streak_count',
        'streakCode_y': 'end_streak_code',
        'streakCount_y':'end_streak_count',
        'winPctg_x': 'tx_win_pctg',
        'winPctg_y':'end_win_pctg',
        'teamName.default':'team_name',
        'teamAbbrev.default':'tri_code'
    })
christmas_clean.head()

In [ ]:
christmas_clean['goal_diff_pct_change'] = christmas_clean['tx_goal_diff_pct']-christmas_clean['end_goal_diff_pct']

In [ ]:
christmas_clean['season_id'] = christmas_clean['season_id'].fillna(0).astype('int64')
christmas_clean['season_end'] = christmas_clean['season_end'].fillna(0).astype('int64')

In [ ]:
christmas_clean['status_a'] = np.where(christmas_clean['thanksgiving'] < 9, 1, 0)
christmas_clean['status_b'] = np.where(christmas_clean['season_end'] < 9, 1, 0)
christmas_clean.head()

In [ ]:
# Filter teams in playoff position at Thanksgiving
playoff_at_xmas = christmas_clean[christmas_clean['status_a'] == 1]

# Calculate odds of making playoffs
odds = playoff_at_xmas['status_b'].mean()  # Mean gives the proportion of 1s

print(f"Odds of making playoffs if in playoff position at Thanksgiving: {odds:.2%}")


In [ ]:
#is there a difference between the conferences?
conference_odds = playoff_at_xmas.groupby('conference')['status_b'].mean()
print(conference_odds)


In [ ]:
# Filter teams not in playoff position at Thanksgiving but made playoffs
late_surge_teams = christmas_clean[(christmas_clean['status_a'] == 0) & (christmas_clean['status_b'] == 1)]

# Calculate distance to 8th spot (positive values indicate better positioning)
late_surge_teams['distance_to_8th'] = 8 - late_surge_teams['thanksgiving']


# Summary statistics on how close they were
avg_distance = late_surge_teams['distance_to_8th'].mean()
median_distance = late_surge_teams['distance_to_8th'].median()
min_distance = late_surge_teams['distance_to_8th'].min()
mean_change_in_goal_diff = late_surge_teams['goal_diff_pct_change'].mean()

print(f"Average distance to 8th spot: {avg_distance:.2f}")
print(f"Median distance to 8th spot: {median_distance:.2f}")
print(f"Closest team to 8th spot: {min_distance:.2f}")
print(f"Avg change in goal diff percentage: {mean_change_in_goal_diff:.2f}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# Apply one-hot encoding to the 'conference' column
X = christmas_clean[['tx_points', 'tx_points_pct', 'tx_goal_diff', 'thanksgiving']]
X = pd.get_dummies(X, drop_first=True)

# Add the encoded 'conference' column
X['conference'] = christmas_clean['conference'].map({'eastern': 0, 'western': 1})

# Define target
y = christmas_clean['status_b']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the random forest
rf = RandomForestClassifier(random_state=42, n_estimators=100)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

# Feature importance
importances = rf.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance:.4f}")


In [ ]:
# Create streak features
def calculate_streak(row, streak_code_col):
    """
    Calculates the streak value based on the streak code column provided.
    W = 1, L = -1, and other cases will be 0 (or missing streak).
    """
    if row[streak_code_col] == 'W':
        return 1  # Win streak is positive
    elif row[streak_code_col] == 'L':
        return -1  # Loss streak is negative
    else:
        return 0  # Handle cases where streak data might be missing or undefined

# Apply the function to calculate Thanksgiving streak
christmas_clean['xmas_streak'] = christmas_clean.apply(
    calculate_streak, axis=1, streak_code_col='tx_streak_code')

# Apply the function to calculate Season-end streak
christmas_clean['season_end_streak'] = christmas_clean.apply(
    calculate_streak, axis=1, streak_code_col='end_streak_code')

# Drop the original streak columns (optional)
christmas_clean = christmas_clean.drop(columns=['tx_streak_code', 'end_streak_code'])

# Check the result
christmas_clean.head()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import xgboost as xgb
from sklearn.metrics import classification_report

# Example of applying one-hot encoding to categorical features (including 'conference')
X = christmas_clean[['tx_points', 'tx_points_pct', 'tx_goal_diff', 'thanksgiving', 'conference']]
X = pd.get_dummies(X, drop_first=True)

# Define target
y = christmas_clean['status_b']  # 1 if made playoffs, 0 otherwise

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)

# Fit the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Feature importance
importances = xgb_model.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance:.4f}")


In [ ]:
# Assuming 'thanksgiving_clean' has columns 'teamName' and 'season_end_winPctg' for season-end points percentage

team_strength = {}

# Iterate through the DataFrame to create the team_strength dictionary
for _, row in christmas_clean.iterrows():
    team_name = row['team_name']  # Team name
    win_pctg = row['end_win_pctg']  # Season-end win percentage (points percentage)

    # Store the team's strength in the dictionary
    team_strength[team_name] = win_pctg

# Check the result
team_strength


In [ ]:

#get games this season

# Initialize the DataFrame
daily_games = pd.DataFrame()

base_url = "https://api-web.nhle.com/v1/schedule/"
# start_date = datetime.strptime("2023-10-09", "%Y-%m-%d")
# end_date = datetime.strptime("2024-04-17", "%Y-%m-%d")

start_date = datetime.strptime("2024-10-04", "%Y-%m-%d")
end_date = datetime.strptime("2024-12-24", "%Y-%m-%d")


current_date = start_date

# Set to keep track of unique dates
seen_dates = set()

while current_date <= end_date:
    # Format the date as 'YYYY-MM-DD'
    formatted_date = current_date.strftime("%Y-%m-%d")
    api_url = f"{base_url}{formatted_date}"
    
    # Make the API request
    response = requests.get(api_url)
    
    if response.status_code == 200:
    # The response content can be accessed using response.text
        response_text = response.text
    #pprint(response_text)
    else:
        print(f"Request failed with status code {response.status_code}")

    json_data = json.loads(response_text)

    game_week = json_data['gameWeek']
    game_week_df = pd.DataFrame(game_week)


    game_week_df = game_week_df[game_week_df['numberOfGames'] != 0]
    
        # Filter out rows with duplicate dates
    if formatted_date not in seen_dates:
            seen_dates.add(formatted_date)
            daily_games = pd.concat([daily_games, game_week_df], ignore_index=True)
    else:
        print(f"Failed to retrieve data for {formatted_date}")
    
    # Move to the next week
    current_date += timedelta(weeks=1)
    # Filter out rows where 'date' is after the end date
    daily_games['date'] = pd.to_datetime(daily_games['date'])
    daily_games = daily_games[daily_games['date'] <= end_date]

    # Reset index after filtering
    daily_games.reset_index(drop=True, inplace=True)

    game_week_details = pd.json_normalize(daily_games['games'])
game_week_details.tail()





#in order to extract the data properly, it needs to be stored in a dictionary. This extracts each of the games from the cells

dfs = {}

# Loop through the iterations (30 times)
for i in range(0, len(game_week_details.columns)): 
    api_response = game_week
    
    if api_response is not None:
        # Extract relevant data from the API response and normalize it
        game_info = pd.json_normalize(game_week_details[i])
        
        # Create a DataFrame for this iteration
        df_name = f'game_test{i}'  # Generate a unique variable name
        dfs[df_name] = pd.DataFrame(game_info)
    else:
        # Handle the case where the API request failed
        print(f"API request failed for index {i}")

#Then I combine all of the dfs in the list by concatenation to create a single df. now all of the game data is spread out across each row. 
combined_df = pd.concat(dfs.values(), ignore_index=True)
combined_df.dropna(how='all', inplace=True)



combined_df = combined_df[['id', 'season', 'startTimeUTC', 'awayTeam.id', 'awayTeam.abbrev', 
                        'homeTeam.id', 'homeTeam.abbrev',
                        'awayTeam.score', 'homeTeam.score']]


combined_df=combined_df.convert_dtypes()
combined_df['id'] = combined_df['id'].astype(str)


# Assuming '<NA>' is a string, replace it with np.nan
combined_df['id'] = combined_df['id'].replace('<NA>', np.nan)

# Drop rows with NaN values in the 'link' column
combined_df = combined_df.dropna(subset=['id'])
combined_df['startTimeUTC'] = pd.to_datetime(combined_df['startTimeUTC'])
combined_df = combined_df.rename(columns = {'id':'game_id'})
combined_df = combined_df.sort_values('game_id').reset_index()


# Specify the UTC time zone
utc_timezone = pytz.utc

# Specify the target time zone (Eastern Time)
eastern_timezone = pytz.timezone('America/New_York')

# Convert 'startTimeUTC' to Eastern Time
combined_df['game_date_time'] = combined_df['startTimeUTC'].dt.tz_convert(eastern_timezone)
combined_df['game_date_time'] = pd.to_datetime(combined_df['game_date_time'])
combined_df['start_time'] =combined_df['game_date_time'].dt.strftime('%I:%M %p').str.lstrip('0').str.lower()
combined_df['game_date'] = combined_df['game_date_time'].dt.strftime('%Y-%m-%d')
combined_df.drop('startTimeUTC', axis=1, inplace=True)
# combined_df = combined_df[combined_df['game_date'] == formatted_date]
combined_df.sort_values(by='game_id')
# print("combined_df done")
combined_df.head()


In [ ]:

# Define season start and thanksgiving dates
season_start_dates = {
    "20002001": "2000-10-04", "20012002": "2001-10-03", "20022003": "2002-10-09", "20032004": "2003-10-08",
    "20052006": "2005-10-05", "20062007": "2006-10-04", "20072008": "2007-09-29", "20082009": "2008-10-04",
    "20092010": "2009-10-01", "20102011": "2010-10-07", "20112012": "2011-10-06", "20122013": "2013-01-19", 
    "20132014": "2013-10-01", "20142015": "2014-10-08", "20152016": "2015-10-07", "20162017": "2016-10-12",
    "20172018": "2017-10-04", "20182019": "2018-10-04", "20192020": "2019-10-02", "20202021": "2021-01-13",  
    "20212022": "2021-10-12", "20222023": "2022-10-11", "20232024": "2023-10-11"
}

christmas_dates = {
    '20002001': '2000-12-24', '20012002': '2001-12-24', '20022003': '2002-12-24', '20032004': '2003-12-24',
    '20052006': '2005-12-24', '20062007': '2006-12-24', '20072008': '2007-12-24', '20082009': '2008-12-24',
    '20092010': '2009-12-24', '20102011': '2010-12-24', '20112012': '2011-12-24', '20122013': '2013-02-13', '20132014': '2013-12-24',
    '20142015': '2013-12-24', '20152016': '2015-12-24', '20162017': '2016-11-24', '20172018': '2017-11-23',
    '20182019': '2018-11-22', '20192020': '2019-11-28', '20202021': '2021-03-04', '20212022': '2021-02-15',
    '20222023': '2022-11-24', '20232024': '2023-11-23', '20242025': '2024-11-28'
}

# Initialize the DataFrame
daily_games = pd.DataFrame()

# Base URL for the API
base_url = "https://api-web.nhle.com/v1/schedule/"

# Loop through each season and get data
for season_id, season_start in season_start_dates.items():
    start_date = datetime.strptime(season_start, "%Y-%m-%d")
    end_date = datetime.strptime(thanksgiving_dates[season_id], "%Y-%m-%d")

    current_date = start_date
    seen_dates = set()

    # Loop through the season by week
    while current_date <= end_date:
        # Format the date as 'YYYY-MM-DD'
        formatted_date = current_date.strftime("%Y-%m-%d")
        api_url = f"{base_url}{formatted_date}"

        # Make the API request
        response = requests.get(api_url)

        if response.status_code == 200:
            response_text = response.text
        else:
            print(f"Request failed for {formatted_date} with status code {response.status_code}")
            current_date += timedelta(weeks=1)
            continue

        # Parse the JSON response
        json_data = json.loads(response_text)

        # Process the game data
        if 'gameWeek' in json_data:
            game_week = json_data['gameWeek']
            game_week_df = pd.DataFrame(game_week)

            # Filter out weeks with no games
            game_week_df = game_week_df[game_week_df['numberOfGames'] != 0]

            # Filter out duplicate dates for the season
            if formatted_date not in seen_dates:
                seen_dates.add(formatted_date)
                game_week_df['season_id'] = season_id
                daily_games = pd.concat([daily_games, game_week_df], ignore_index=True)
            else:
                print(f"Duplicate data found for {formatted_date}")
        else:
            print(f"No 'gameWeek' data found for {formatted_date}")

        # Move to the next week
        current_date += timedelta(weeks=1)

# Convert the 'date' column to datetime format
daily_games['date'] = pd.to_datetime(daily_games['date'])
daily_games = daily_games[daily_games['date'] <= end_date]

# Reset index
daily_games.reset_index(drop=True, inplace=True)

# Normalize the 'games' data if it exists
if 'games' in daily_games.columns:
    game_week_details = pd.json_normalize(daily_games['games'])
else:
    print("No 'games' column found. Please check the data structure.")

# Create a dictionary for the extracted data
dfs = {}

# Loop through the iterations (game details)
for i in range(len(game_week_details.columns)):
    api_response = game_week_details[i]

    if api_response is not None:
        game_info = pd.json_normalize(api_response)
        df_name = f'game_test{i}'  # Unique name for each game
        dfs[df_name] = pd.DataFrame(game_info)
    else:
        print(f"API request failed for index {i}")

# Combine all the game data into a single DataFrame
combined_df = pd.concat(dfs.values(), ignore_index=True)
combined_df.dropna(how='all', inplace=True)

# Filter relevant columns
combined_df = combined_df[['id', 'season', 'startTimeUTC', 'awayTeam.id', 'awayTeam.abbrev', 
                           'homeTeam.id', 'homeTeam.abbrev', 'awayTeam.score', 'homeTeam.score']]

# Convert data types
combined_df = combined_df.convert_dtypes()
combined_df['id'] = combined_df['id'].astype(str)

# Handle missing or invalid data
combined_df['id'] = combined_df['id'].replace('<NA>', pd.NA)

# Drop rows with NaN values in the 'id' column
combined_df = combined_df.dropna(subset=['id'])

# Convert 'startTimeUTC' to datetime
combined_df['startTimeUTC'] = pd.to_datetime(combined_df['startTimeUTC'])

# Convert to Eastern Time
utc_timezone = pytz.utc
eastern_timezone = pytz.timezone('America/New_York')

combined_df['game_date_time'] = combined_df['startTimeUTC'].dt.tz_convert(eastern_timezone)
combined_df['game_date_time'] = pd.to_datetime(combined_df['game_date_time'])

# Extract time and date details
combined_df['start_time'] = combined_df['game_date_time'].dt.strftime('%I:%M %p').str.lstrip('0').str.lower()
combined_df['game_date'] = combined_df['game_date_time'].dt.strftime('%Y-%m-%d')

# Clean up the final DataFrame
combined_df.drop('startTimeUTC', axis=1, inplace=True)
combined_df.sort_values(by='id', inplace=True)
combined_df.reset_index(drop=True, inplace=True)

# Output the final combined DataFrame (this can be called later)
final_df = combined_df

# Display the final DataFrame (optional)
final_df.head()
